In [2]:
# generate split from root

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tqdm
import glob

np.random.seed(seed=2358)

%matplotlib inline

# os.chdir("/home/data/workspace/heqi/monogastroendo/")

root="/home/data/workspace/heqi/monogastroendo/rect_c3vd_data/"

folders = glob.glob(root+"*")
folders.sort()
splits = [['cecum_t1_a_under_review/c1v1',
            'cecum_t1_b_under_review/c1v3',
            'cecum_t2_a_under_review/c2v1',
            'cecum_t2_b_under_review/c2v2',
            'cecum_t2_c_under_review/c2v3',
            'cecum_t3_a_under_review/c3v2',
            'desc_t4_a_under_review/d4v2',
            'sigmoid_t1_a_under_review/s1v3',
            'sigmoid_t2_a_under_review/s2v1',
            'trans_t1_a_under_review/t1v1',
            'trans_t1_b_under_review/t1v3',
            'trans_t2_a_under_review/t2v1',
            'trans_t2_b_under_review/t2v2',
            'trans_t2_c_under_review/t2v3',
            'trans_t3_a_under_review/t3v2',
            'trans_t3_b_under_review/t3v3',],
          ['cecum_t4_a_under_review/c4v2',
            'cecum_t4_b_under_review/c4v3',
            'sigmoid_t3_a_under_review/s3v1',
            'sigmoid_t3_b_under_review/s3v2',
            'trans_t4_a_under_review/t4v1',
            'trans_t4_b_under_review/t4v3']]

def check_valid_input(splits):
    for split in splits:
        for folder in split:
            src = os.path.join(root, folder)
            assert(os.path.exists(src))

def load_split(folders, skip=5):
    outputs = []
    for folder in folders:
        files = glob.glob(root+folder+"/*_color.png")
        for i in range(0, len(files) - 2*skip):
            line = [folder]
            for j in range(3):
                line.append(f"{i+j*skip}")
                fname = f"{root+folder}/{i+j*skip}_color.png"
                assert(os.path.exists(fname))
            outputs.append(" ".join(line))
    return outputs

def check_opticflow(prvs, next):
    import cv2
    prvs = cv2.cvtColor(prvs, cv2.COLOR_BGR2GRAY)
    # Convert the frame to grayscale
    next = cv2.cvtColor(next, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow using the Lucas-Kanade method
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    return flow


check_valid_input(splits)

train = load_split(splits[0])
test = load_split(splits[1])

with open("train_files.txt", "w") as f:
    for line in train:
        f.write(line+"\n")

with open("val_files.txt", "w") as f:
    for line in test:
        f.write(line+"\n")

In [20]:
import cv2
import numpy as np

def check_opticflow(prvs, next):
    prvs = cv2.cvtColor(prvs, cv2.COLOR_BGR2GRAY)
    # Convert the frame to grayscale
    next = cv2.cvtColor(next, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow using the Lucas-Kanade method
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    return flow

def test_check_opticflow():
    root = "/home/data/workspace/heqi/monogastroendo/rect_c3vd_data/"
    folder = "cecum_t1_a_under_review/c1v1"
    skip=16
    i=0
    prvs = f"{root+folder}/{i+0*skip}_color.png"
    next = f"{root+folder}/{i+1*skip}_color.png"
    prvs = cv2.imread(prvs)
    next = cv2.imread(next)
    flow = check_opticflow(prvs, next)
    
    return flow

flow = test_check_opticflow()

print(flow.max(), flow.min(), np.abs(flow).mean(), flow.shape)

28.180983 -23.331966 0.42609474 (1080, 1350, 2)


In [19]:
import cv2
import numpy as np
import torch

import os
os.chdir("/home/data/workspace/heqi/monogastroendo/")
from layers import LoFTR
import kornia as K
import kornia.feature as KF
from kornia_moons.viz import draw_LAF_matches


def load_torch_image(fname):
    img = K.image_to_tensor(cv2.resize(cv2.imread(fname), (320, 256)), False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

def check_correspondences(prvs, next, net=None, show=True):
    prvs = load_torch_image(prvs).to(device)
    next = load_torch_image(next).to(device)
    cor = net(prvs, next)

    if show:
        mkpts0 = cor['keypoints0'].cpu().numpy()
        mkpts1 = cor['keypoints1'].cpu().numpy()

        print("kp0 has {} key points".format(mkpts0.shape[0]))
        print("kp1 has {} key points".format(mkpts1.shape[0]))

        draw_LAF_matches(
            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1,-1, 2),
                                        torch.ones(mkpts0.shape[0]).view(1,-1, 1, 1),
                                        torch.ones(mkpts0.shape[0]).view(1,-1, 1)),

            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1,-1, 2),
                                        torch.ones(mkpts1.shape[0]).view(1,-1, 1, 1),
                                        torch.ones(mkpts1.shape[0]).view(1,-1, 1)),
            torch.arange(mkpts0.shape[0]).view(-1,1).repeat(1,2),
            K.tensor_to_image(prvs),
            K.tensor_to_image(next),
            draw_dict={'inlier_color': (0.2, 1, 0.2),
                    'tentative_color': None, 
                    'feature_color': (0.2, 0.5, 1), 'vertical': False})
    
    return cor

def test_check_correspondences(net):
    root = "/home/data/workspace/heqi/monogastroendo/rect_c3vd_data/"
    folder = "cecum_t1_a_under_review/c1v1"
    i=0
    prvs = f"{root+folder}/{i+0*skip}_color.png"
    next = f"{root+folder}/{i+1*skip}_color.png"
    cor = check_correspondences(prvs, next, net=net, show=False)
    
    # print(cor)
    mask = cor['confidence']>0.9
    dist = cor['keypoints0'][mask] - cor['keypoints1'][mask]
    # print(dist.mean())
    
    return cor


def check_dist(net):
    root = "/home/data/workspace/heqi/monogastroendo/rect_c3vd_data/"
    folder = "cecum_t1_a_under_review/c1v1"
    for i in range(100):
        prvs = f"{root+folder}/{i+0*skip}_color.png"
        next = f"{root+folder}/{i+1*skip}_color.png"
        cor = check_correspondences(prvs, next, net=net, show=False)
        
        mask = cor['confidence']>0.9
        dist = cor['keypoints0'][mask] - cor['keypoints1'][mask]
        
        print(dist.abs().mean())
    

skip=20
device = torch.device("cuda")
try:
    net
    print('net exists')
except NameError:
    net = LoFTR().to(device)
    print('net does not exist')
    
correspondences = test_check_correspondences(net)
check_dist(net)


net exists
tensor(8.3037, device='cuda:0')
tensor(8.2939, device='cuda:0')
tensor(7.7998, device='cuda:0')
tensor(8.2927, device='cuda:0')
tensor(8.2553, device='cuda:0')
tensor(8.2507, device='cuda:0')
tensor(8.5285, device='cuda:0')
tensor(9.2108, device='cuda:0')
tensor(9.1012, device='cuda:0')
tensor(8.1310, device='cuda:0')
tensor(7.8676, device='cuda:0')
tensor(8.6148, device='cuda:0')
tensor(8.0486, device='cuda:0')
tensor(8.4997, device='cuda:0')
tensor(8.8880, device='cuda:0')
tensor(8.4974, device='cuda:0')
tensor(8.7938, device='cuda:0')
tensor(8.1053, device='cuda:0')
tensor(8.2620, device='cuda:0')
tensor(8.2682, device='cuda:0')
tensor(8.3271, device='cuda:0')
tensor(7.8326, device='cuda:0')
tensor(7.9617, device='cuda:0')
tensor(7.5511, device='cuda:0')
tensor(7.5976, device='cuda:0')
tensor(7.2175, device='cuda:0')
tensor(7.2931, device='cuda:0')
tensor(7.3074, device='cuda:0')
tensor(7.3691, device='cuda:0')
tensor(7.1795, device='cuda:0')
tensor(6.7509, device='cuda:0

In [18]:
import cv2
import numpy as np
import torch

import os
os.chdir("/home/data/workspace/heqi/monogastroendo/")
from layers import LoFTR
import kornia as K
import kornia.feature as KF
from kornia_moons.viz import draw_LAF_matches

# generate split from root

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tqdm
import glob

np.random.seed(seed=2358)

%matplotlib inline

# os.chdir("/home/data/workspace/heqi/monogastroendo/")

root="/home/data/workspace/heqi/monogastroendo/rect_c3vd_data/"

folders = glob.glob(root+"*")
folders.sort()
splits = [['cecum_t1_a_under_review/c1v1',
            'cecum_t1_b_under_review/c1v3',
            'cecum_t2_a_under_review/c2v1',
            'cecum_t2_b_under_review/c2v2',
            'cecum_t2_c_under_review/c2v3',
            'cecum_t3_a_under_review/c3v2',
            'desc_t4_a_under_review/d4v2',
            'sigmoid_t1_a_under_review/s1v3',
            'sigmoid_t2_a_under_review/s2v1',
            'trans_t1_a_under_review/t1v1',
            'trans_t1_b_under_review/t1v3',
            'trans_t2_a_under_review/t2v1',
            'trans_t2_b_under_review/t2v2',
            'trans_t2_c_under_review/t2v3',
            'trans_t3_a_under_review/t3v2',
            'trans_t3_b_under_review/t3v3',],
          ['cecum_t4_a_under_review/c4v2',
            'cecum_t4_b_under_review/c4v3',
            'sigmoid_t3_a_under_review/s3v1',
            'sigmoid_t3_b_under_review/s3v2',
            'trans_t4_a_under_review/t4v1',
            'trans_t4_b_under_review/t4v3']]

def check_valid_input(splits):
    for split in splits:
        for folder in split:
            src = os.path.join(root, folder)
            assert(os.path.exists(src))


def load_torch_image(fname):
    img = K.image_to_tensor(cv2.resize(cv2.imread(fname), (320, 256)), False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

def check_correspondences(prvs, next, show=True):
    prvs = load_torch_image(prvs).to(device)
    next = load_torch_image(next).to(device)
    cor = net(prvs, next)

    if show:
        mkpts0 = cor['keypoints0'].cpu().numpy()
        mkpts1 = cor['keypoints1'].cpu().numpy()

        print("kp0 has {} key points".format(mkpts0.shape[0]))
        print("kp1 has {} key points".format(mkpts1.shape[0]))

        draw_LAF_matches(
            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1,-1, 2),
                                        torch.ones(mkpts0.shape[0]).view(1,-1, 1, 1),
                                        torch.ones(mkpts0.shape[0]).view(1,-1, 1)),

            KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1,-1, 2),
                                        torch.ones(mkpts1.shape[0]).view(1,-1, 1, 1),
                                        torch.ones(mkpts1.shape[0]).view(1,-1, 1)),
            torch.arange(mkpts0.shape[0]).view(-1,1).repeat(1,2),
            K.tensor_to_image(prvs),
            K.tensor_to_image(next),
            draw_dict={'inlier_color': (0.2, 1, 0.2),
                    'tentative_color': None, 
                    'feature_color': (0.2, 0.5, 1), 'vertical': False})
    
    return cor

def check_dist(prvs, next):
    cor = check_correspondences(prvs, next, show=False)        
    mask = cor['confidence']>0.5
    dist = cor['keypoints0'][mask] - cor['keypoints1'][mask]

    return dist.abs().mean()
    

def load_split(folders):
    outputs = []
    # static_frames = 0
    for folder in folders:
        files = glob.glob(root+folder+"/*_color.png")
        for i in range(0, len(files) - 2*skip):
            line = [folder]
            fnames = []
            for j in range(3):
                line.append(f"{i+j*skip}")
                fname = f"{root+folder}/{i+j*skip}_color.png"
                fnames.append(fname)
                assert(os.path.exists(fname))
            # # skip static frames
            # if check_dist(fnames[0], fnames[1]) < 1 or check_dist(fnames[1], fnames[2]) < 1:
            #     static_frames += 1
            #     continue
            outputs.append(" ".join(line))
    # print("We filterd {} static frames".format(static_frames))
    return outputs

def filter_staticframe(lines):
    static_frames = 0
    outputs = []
    for line in tqdm.tqdm(lines):
        folder = line.split(" ")[0]
        findex = line.split(" ")[1:]
        fnames = []
        for j in range(3):
            fname = f"{root+folder}/{findex[j]}_color.png"
            fnames.append(fname)
            assert(os.path.exists(fname))
        # skip static frames
        if check_dist(fnames[0], fnames[1]) < 1 or check_dist(fnames[1], fnames[2]) < 1:
            static_frames += 1
            # assert(static_frames / len(lines) < 0.2)
        else:
            outputs.append(line)
    print("We filterd {} static frames".format(static_frames))
    return outputs

skip=20
device = torch.device("cuda")
try:
    net
    print('net exists')
except NameError:
    net = LoFTR().to(device)
    print('net does not exist')


check_valid_input(splits)

train = load_split(splits[0])
train = filter_staticframe(train)
test = load_split(splits[1])
test = filter_staticframe(test)

with open("train_files.txt", "w") as f:
    for line in train:
        f.write(line+"\n")

with open("val_files.txt", "w") as f:
    for line in test:
        f.write(line+"\n")
        

net exists


100%|██████████| 6457/6457 [13:06<00:00,  8.21it/s]


We filterd 1825 static frames


100%|██████████| 2778/2778 [05:45<00:00,  8.05it/s]

We filterd 557 static frames
